# Comprehensive SyftBox Permissions Tutorial
## Part 1: Fundamentals and Core Concepts

Welcome to the comprehensive guide to SyftBox permissions! This tutorial series will take you from beginner to expert, covering every aspect of how SyftBox's permission system works.

**The Problem This Tutorial Solves**: SyftBox's permission system is incredibly powerful but complex. Many developers struggle with understanding how permissions flow through directory hierarchies, why certain users can or cannot access files, and how to debug permission issues. This leads to either overly permissive setups (security risk) or overly restrictive ones (blocking legitimate access).

**Our Solution**: A systematic walkthrough of every permission concept with hands-on examples, real debugging scenarios, and clear explanations of the underlying algorithms. By the end, you'll understand not just *how* to set permissions, but *why* they work the way they do.

### What You'll Learn in This Series

1. **Part 1 (This notebook): Fundamentals** - Core concepts, permission levels, basic inheritance
2. **Part 2: Patterns and Matching** - Glob patterns, specificity, wildcards, and double-star patterns  
3. **Part 3: Inheritance and Hierarchy** - How permissions flow through directory trees
4. **Part 4: Terminal Nodes and Blocking** - Advanced inheritance control
5. **Part 5: File Limits and Restrictions** - Size limits, file type controls
6. **Part 6: Complex Scenarios** - Real-world edge cases and debugging

### Prerequisites
- Basic Python knowledge
- Familiarity with file systems and directories  
- Have completed the "5-minute quickstart" tutorial

## Setup

In [ ]:
import syft_perm
from pathlib import Path
import tempfile
import shutil
import yaml

# Create a clean workspace for this tutorial
tutorial_dir = Path(tempfile.mkdtemp(prefix="syftbox_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    """Clean up and create fresh workspace"""
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="syftbox_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path):
    """Display the generated syft.pub.yaml file"""
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        print(f"\n=== {yaml_file} ===")
        print(yaml_file.read_text())
        print("=" * 50)
    else:
        print(f"No syft.pub.yaml in {path}")

## Chapter 1: Understanding Permission Levels

**The Problem**: In most systems, permissions are either "all or nothing" - you either have access or you don't. But in collaborative data science, you need nuanced control. A research collaborator might need to read your data but not modify it. A data engineer might need to create new files but not delete existing ones. A project lead might need full control.

**The Solution**: SyftBox uses a **4-level permission hierarchy** where each level builds on the previous one. This gives you precise control over what each collaborator can do.

Let's explore these levels and see how they build on each other:

In [ ]:
# Create a test file
test_file = tutorial_dir / "example.txt"
test_file.write_text("Example content")

syft_file = syft_perm.open(test_file)

# The 4 permission levels (lowest to highest)
print("=== Permission Hierarchy ===")
print("1. READ - Can view file contents")
print("2. CREATE - Can read + create new files")
print("3. WRITE - Can read + create + modify existing files")
print("4. ADMIN - Can read + create + write + manage permissions")
print("\nHigher levels include all lower level permissions!")

In [ ]:
# Demonstrate permission hierarchy - this is the key insight!
syft_file.grant_write_access("alice@example.com", force=True)

print("🔑 KEY INSIGHT: Permission levels are cumulative!")
print("Alice has WRITE permission, which automatically gives her:")
print(f"- Read access: {syft_file.has_read_access('alice@example.com')}")
print(f"- Create access: {syft_file.has_create_access('alice@example.com')}")
print(f"- Write access: {syft_file.has_write_access('alice@example.com')}")
print(f"- Admin access: {syft_file.has_admin_access('alice@example.com')}")

print("\n💡 Why this matters:")
print("- You don't need to grant Read AND Write - Write includes Read")
print("- This prevents common mistakes where users have Write but not Read")
print("- It simplifies permission management in complex hierarchies")

syft_file

## Chapter 2: The syft.pub.yaml File Structure

**The Problem**: Many permission systems store access rules in databases or binary formats that are hard to inspect, version control, or understand. When things go wrong, it's nearly impossible to debug what permissions are actually set.

**The Solution**: SyftBox stores all permissions in human-readable YAML files called `syft.pub.yaml`. These files live alongside your data, can be version controlled with git, and are easy to inspect and understand.

Let's decode what syft-perm generates behind the scenes:

In [ ]:
# Look at the generated YAML - this is what's actually stored on disk
show_yaml(tutorial_dir)

print("\n🔍 YAML Structure Breakdown:")
print("rules:                    # List of permission rules (can have multiple)")
print("- pattern: 'example.txt'  # Which files this rule applies to (glob patterns)")
print("  access:                 # Permission grants for this pattern")
print("    write:                # Permission level (read/create/write/admin)")
print("    - alice@example.com   # Users with this permission")

print("\n💡 Key Benefits:")
print("✅ Human readable - you can inspect permissions by eye")
print("✅ Version controllable - track permission changes in git")
print("✅ Portable - copy files and permissions move with them") 
print("✅ Debuggable - see exactly what rules are active")
print("✅ Editable - advanced users can edit YAML directly")

## Chapter 3: Multiple Users and Permission Levels

**The Problem**: Real projects involve teams with different roles. Your data science project might have:
- External reviewers who need read-only access  
- Junior researchers who can create new analyses but shouldn't modify core data
- Senior researchers who can modify existing work
- Project leads who manage the entire permission structure

**The Solution**: SyftBox lets you grant different permission levels to different users on the same file. The permission hierarchy ensures everyone gets exactly the access they need without over-privileging.

Let's build a realistic team permission structure:

In [ ]:
# Grant different permissions to different users - building a realistic team
syft_file.grant_read_access("reviewer@external.org", force=True)      # External reviewer  
syft_file.grant_create_access("junior@university.edu", force=True)    # Junior researcher
syft_file.grant_write_access("senior@university.edu", force=True)     # Senior researcher (alice from before)
syft_file.grant_admin_access("pi@university.edu", force=True)         # Principal investigator

print("🏗️ Building Realistic Team Permissions:")
print("👀 External Reviewer - Read only (can review but not modify)")
print("🔰 Junior Researcher - Create access (can add new files)")  
print("✏️  Senior Researcher - Write access (can modify existing work)")
print("👑 Principal Investigator - Admin access (manages permissions)")

print("\n=== Permission Matrix ===")
users = [
    ("reviewer@external.org", "External Reviewer"),
    ("junior@university.edu", "Junior Researcher"), 
    ("senior@university.edu", "Senior Researcher"),
    ("pi@university.edu", "Principal Investigator")
]
permissions = ["read", "create", "write", "admin"]

print(f"{'Role':<20} {'Read':<6} {'Create':<8} {'Write':<6} {'Admin':<6}")
print("-" * 55)

for email, role in users:
    row = f"{role:<20}"
    for perm in permissions:
        has_perm = getattr(syft_file, f"has_{perm}_access")(email)
        row += f" {'✓' if has_perm else '✗':<6}"
    print(row)

print("\n💡 Notice the hierarchy in action:")
print("- PI (admin) can do everything")
print("- Senior (write) can read, create, and write")  
print("- Junior (create) can read and create")
print("- Reviewer (read) can only read")

syft_file

In [ ]:
# Look at the updated YAML - notice how permissions accumulate efficiently
show_yaml(tutorial_dir)

print("🔑 Key Insights from the YAML:")
print("1. Each permission level gets its own list in the YAML")
print("2. Users are only listed at their HIGHEST permission level")
print("3. The hierarchy is handled by the syft-perm algorithm, not the YAML")
print("4. This keeps the YAML clean and prevents redundancy")
print("\n💡 Why this matters:")
print("- Easier to understand who has what level of access")
print("- No redundant entries (PI isn't listed under read, create, write)")
print("- Permission changes are atomic (change one line, not four)")

## Chapter 4: Public Access with "*"

**The Problem**: Sometimes you want to share data publicly - like publishing research results, sharing open datasets, or creating community resources. Traditional permission systems make this awkward by requiring you to explicitly list every possible user.

**The Solution**: SyftBox uses the special `"*"` user to mean "everyone". This is conceptually similar to making a file "world-readable" in Unix, but with the same fine-grained permission controls.

Let's explore how public access works and when to use it:

In [ ]:
# Create a public file - imagine this is a published research dataset
cleanup_and_reset()
public_file = tutorial_dir / "published_results.csv"
public_file.write_text("year,discovery,impact_factor\n2023,Novel ML Algorithm,8.5\n2024,Quantum Breakthrough,9.2")

syft_public = syft_perm.open(public_file)
syft_public.grant_read_access("*", force=True)  # Everyone can read

print("📢 Creating Public Research Dataset")
print("✅ Published results should be readable by anyone")
print("✅ Using '*' makes this explicit and maintainable")

# Test with random users - simulating researchers from around the world accessing this data
test_users = [
    "researcher@mit.edu", 
    "student@stanford.edu", 
    "scientist@oxford.ac.uk",
    "anyone@anywhere.com"
]

print("\n🌍 Global Access Test (simulating researchers worldwide):")
for user in test_users:
    can_read = syft_public.has_read_access(user)
    status = "✅ Can access" if can_read else "❌ No access"
    print(f"{user}: {status}")

print("\n💡 Real-world use cases for public access:")
print("📊 Published datasets")
print("📄 Research papers and documentation") 
print("🔧 Open-source tools and utilities")
print("📈 Public dashboards and visualizations")

show_yaml(tutorial_dir)

## Chapter 5: Folder Permissions and Basic Inheritance

**The Problem**: In real projects, you don't want to set permissions on individual files one by one. You have directories full of related files (datasets, analyses, documentation) that should all have the same access control. Managing hundreds of individual file permissions would be a nightmare.

**The Solution**: SyftBox lets you set permissions on folders, and files automatically inherit those permissions from their parent directories. This follows the principle of "least surprise" - if you can access a folder, you expect to access the files inside it.

Let's see inheritance in action with a realistic project structure:

In [ ]:
# Create a realistic data science project structure
cleanup_and_reset()
project_dir = tutorial_dir / "climate_analysis_2024"
project_dir.mkdir()

# Create a typical data science project structure
files = {
    "README.md": "# Climate Analysis Project\nAnalyzing temperature trends...",
    "main.py": "import pandas as pd\n# Main analysis script",
    "data.csv": "date,temperature,location\n2024-01-01,15.2,NYC",
    "results.json": '{"avg_temp": 16.8, "trend": "increasing"}',
    "requirements.txt": "pandas>=1.5.0\nmatplotlib>=3.6.0"
}

for filename, content in files.items():
    filepath = project_dir / filename
    filepath.write_text(content)

print("🏗️ Created Realistic Data Science Project:")
print(f"📁 {project_dir.name}/")
for filename in files.keys():
    print(f"   📄 {filename}")

print(f"\n💡 Scenario: You have {len(files)} files that should all have the same permissions")
print("❌ Bad approach: Set permissions on each file individually (5 operations)")
print("✅ Good approach: Set permissions on the folder (1 operation, files inherit)")

In [ ]:
# Apply permissions to the FOLDER - this is the key efficiency gain
syft_folder = syft_perm.open(project_dir)

# Set up realistic collaborator permissions
syft_folder.grant_read_access("collaborator@partner.org", force=True)    # External collaborator can read
syft_folder.grant_write_access("researcher@myuniversity.edu", force=True) # Internal researcher can modify

print("⚡ EFFICIENCY: Set permissions once on the folder")
print("👥 External collaborator gets read access to entire project")
print("🔬 Internal researcher gets write access to entire project")
print("\n📋 Folder permissions:")
syft_folder

In [ ]:
# 🎯 THE MAGIC: Check that ALL files automatically inherit the folder's permissions
print("🔮 INHERITANCE MAGIC: Files automatically inherit folder permissions")
print("\n=== Inheritance Verification ===")

for filename in files.keys():
    filepath = project_dir / filename
    syft_file = syft_perm.open(filepath)
    
    collab_read = syft_file.has_read_access("collaborator@partner.org")
    researcher_write = syft_file.has_write_access("researcher@myuniversity.edu")
    
    status_collab = "✅" if collab_read else "❌"
    status_researcher = "✅" if researcher_write else "❌" 
    
    print(f"📄 {filename}:")
    print(f"   {status_collab} Collaborator can read: {collab_read}")
    print(f"   {status_researcher} Researcher can write: {researcher_write}")

print("\n🎊 ALL FILES inherited permissions without any individual configuration!")
print("💡 Benefits:")
print("   ⚡ Massive time savings - configure once, apply to many")
print("   🔒 Consistent security - no files accidentally left unprotected")
print("   🧹 Easy maintenance - change folder permissions to update everything")

In [ ]:
# Look at the YAML - only the folder has rules, files inherit
show_yaml(tutorial_dir)
show_yaml(project_dir)  # Folder has the rules

## Chapter 6: Permission Debugging and Reasoning

**The Problem**: When permissions don't work as expected, debugging can be a nightmare. Why does Alice have access to this file but not that one? Where are the permissions coming from? Traditional systems give you cryptic error messages or no explanation at all.

**The Solution**: SyftBox includes powerful debugging tools that explain exactly WHY a permission was granted or denied. These tools trace the nearest-node algorithm step-by-step and show you the exact reasoning chain.

**When You Need This**: 
- Permissions aren't working as expected
- Auditing who has access to what
- Understanding complex inheritance patterns
- Debugging team collaboration issues

Let's explore the debugging tools that will save you hours of frustration:

In [ ]:
# Let's debug the inheritance we just saw
readme_file = syft_perm.open(project_dir / "README.md")

print("=== Detailed Permission Analysis ===")
explanation = readme_file.explain_permissions("team@example.com")
print(explanation)

In [ ]:
# Check specific permission with reasons
has_write, reasons = readme_file._check_permission_with_reasons("team@example.com", "write")
print(f"Team can write README.md: {has_write}")
print("Reasons:")
for reason in reasons:
    print(f"  - {reason}")

## Chapter 7: The Nearest-Node Algorithm

**The Problem**: In complex directory structures, you might have permissions set at multiple levels - on great-grandparent folders, parent folders, and individual files. Which permissions should apply? How do they combine? This question becomes critical in large projects with nested collaborations.

**The Solution**: SyftBox uses the **Nearest-Node Algorithm** - one of the most important concepts in the entire permission system. Instead of accumulating permissions from multiple levels, it finds the **single closest parent directory** that has matching rules and uses those rules exclusively.

**Why This Matters**: This algorithm prevents "permission explosion" where permissions from many levels combine unexpectedly. It's predictable, debuggable, and matches most users' mental model of "the closest permission wins."

Let's see this algorithm in action with a realistic multi-level scenario:

In [ ]:
# Create a realistic university research hierarchy
cleanup_and_reset()

# Realistic scenario: University/Department/Lab/Project structure
# university_research/
#   ├── department_chemistry/          <- Department level
#   │   ├── lab_organic/              <- Lab level  
#   │   │   ├── project_catalysis/    <- Project level
#   │   │   │   └── experiment.txt    <- Deep file
#   │   │   └── lab_notes.txt         <- Lab file
#   │   └── department_policy.txt     <- Department file
#   └── university_handbook.txt       <- University file

university_dir = tutorial_dir / "university_research"
dept_dir = university_dir / "department_chemistry"
lab_dir = dept_dir / "lab_organic"
project_dir = lab_dir / "project_catalysis"

# Create the directory structure
project_dir.mkdir(parents=True)

# Create files at different levels of the hierarchy
files = {
    "university_handbook.txt": university_dir / "university_handbook.txt",
    "department_policy.txt": dept_dir / "department_policy.txt", 
    "lab_notes.txt": lab_dir / "lab_notes.txt",
    "experiment.txt": project_dir / "experiment.txt"
}

for description, path in files.items():
    path.write_text(f"Content of {description}")

print("🏛️ Created Realistic University Research Hierarchy:")
print(f"{university_dir.name}/")
print(f"├── department_chemistry/")
print(f"│   ├── lab_organic/")
print(f"│   │   ├── project_catalysis/")
print(f"│   │   │   └── experiment.txt        <- 4 levels deep!")
print(f"│   │   └── lab_notes.txt")
print(f"│   └── department_policy.txt")
print(f"└── university_handbook.txt")

print("\n💭 Mental Model: Where should each file get its permissions from?")
print("🤔 The nearest-node algorithm will tell us!")

In [ ]:
# Set permissions ONLY on the department level - this is key to understanding the algorithm
syft_dept = syft_perm.open(dept_dir)
syft_dept.grant_read_access("alice@chemistry.edu", force=True)

print("🎯 CRITICAL SETUP: Set permissions only on department_chemistry/")
print("✅ Alice gets read access to department-level directory")
print("❌ NO permissions set on university, lab, or project levels")
print("🔍 Let's see what the nearest-node algorithm discovers...")

show_yaml(dept_dir)
print("💡 Only ONE directory in the entire hierarchy has permission rules!")

In [ ]:
# 🧠 THE NEAREST-NODE ALGORITHM IN ACTION
print("🔍 NEAREST-NODE ALGORITHM ANALYSIS")
print("For each file, we trace UP the directory tree to find the nearest syft.pub.yaml with matching rules:")
print()

for description, path in files.items():
    syft_file = syft_perm.open(path)
    can_read = syft_file.has_read_access("alice@chemistry.edu")
    
    # Calculate relative path for clarity
    relative_path = path.relative_to(tutorial_dir)
    
    print(f"📄 {description} ({relative_path})")
    print(f"   🔍 Searching UP: {path.name} → {path.parent.name} → ... → department_chemistry/")
    
    if can_read:
        print(f"   ✅ Alice CAN read: Found permissions in department_chemistry/")
        print(f"   🎯 Nearest node: department_chemistry/ (2-3 levels up)")
    else:
        print(f"   ❌ Alice CANNOT read: No permissions found in any parent")
        print(f"   🎯 Nearest node: None")
    
    # Show the detailed reasoning
    has_read, reasons = syft_file._check_permission_with_reasons("alice@chemistry.edu", "read")
    if reasons:
        print(f"   💡 Reason: {reasons[0]}")
    print()

print("🧠 ALGORITHM INSIGHTS:")
print("1. 🔍 Each file searches UP its directory tree")
print("2. 🎯 Finds the NEAREST directory with a syft.pub.yaml containing matching rules") 
print("3. 🚫 Ignores any other directories (no accumulation)")
print("4. ✅ Uses that single directory's rules exclusively")

print("\n💡 Why 'experiment.txt' (deepest file) gets permissions:")
print("   - Searches: project_catalysis/ → lab_organic/ → department_chemistry/")
print("   - Finds rules in department_chemistry/ → Uses those rules")
print("   - Ignores deeper directories that have no rules")

**Key Insight**: 
- `file.txt` and `sibling.txt` inherit from `parent/` (nearest node with rules)
- `other.txt` has no permissions because there's no matching rule in any parent

## Chapter 8: Ownership and Automatic Permissions

File owners automatically get all permissions, regardless of explicit rules.

In [ ]:
# Create a datasite structure (simulates SyftBox ownership)
cleanup_and_reset()

# SyftBox determines ownership by path: datasites/{email}/...
datasite_dir = tutorial_dir / "datasites" / "alice@example.com"
datasite_dir.mkdir(parents=True)

# Alice's file in her datasite
alice_file = datasite_dir / "my_file.txt"
alice_file.write_text("Alice's private data")

syft_alice_file = syft_perm.open(alice_file)

# Test ownership detection
print("=== Ownership Test ===")
print(f"Alice owns her file: {syft_alice_file.has_admin_access('alice@example.com')}")
print(f"Bob can't access: {syft_alice_file.has_read_access('bob@example.com')}")

# Show reasoning
has_admin, reasons = syft_alice_file._check_permission_with_reasons("alice@example.com", "admin")
print(f"\nWhy Alice has admin access: {reasons}")

## Chapter 9: Revoking Permissions

In [ ]:
# Create a file with multiple users
cleanup_and_reset()
shared_file = tutorial_dir / "shared.txt"
shared_file.write_text("Shared content")

syft_shared = syft_perm.open(shared_file)

# Grant permissions to multiple users
users = ["alice@example.com", "bob@example.com", "charlie@example.com"]
for user in users:
    syft_shared.grant_read_access(user, force=True)

print("Before revocation:")
syft_shared

In [ ]:
# Revoke Bob's access
syft_shared.revoke_read_access("bob@example.com")

print("After revoking Bob's access:")
for user in users:
    can_read = syft_shared.has_read_access(user)
    print(f"{user}: {can_read}")

syft_shared

In [ ]:
# Revoke public access
syft_shared.grant_read_access("*", force=True)  # Grant public access first
print("After granting public access:")
print(f"Random user can read: {syft_shared.has_read_access('random@example.com')}")

syft_shared.revoke_read_access("*")  # Revoke public access
print("\nAfter revoking public access:")
print(f"Random user can read: {syft_shared.has_read_access('random@example.com')}")
print(f"Alice still can read: {syft_shared.has_read_access('alice@example.com')}")

## Summary: Part 1 Key Concepts

🎉 **Congratulations!** You've mastered the fundamental concepts that form the foundation of SyftBox permissions.

### ✅ **Core Concepts You Now Understand**

1. **🏗️ Permission Hierarchy** - The cumulative nature of Read < Create < Write < Admin
   - **Why it matters**: Prevents over-privileging and simplifies management
   - **Real impact**: Grant Write once instead of Read + Create + Write separately

2. **📄 YAML Storage** - Human-readable, version-controllable permission files
   - **Why it matters**: You can inspect, debug, and version control permissions
   - **Real impact**: Permissions move with your data and are transparent

3. **👥 Multi-User Management** - Different roles get different permission levels
   - **Why it matters**: Supports realistic team collaboration patterns
   - **Real impact**: External reviewers, junior researchers, senior staff all get appropriate access

4. **🌐 Public Access** - Use `"*"` for "everyone" permissions
   - **Why it matters**: Easy sharing of public datasets and results
   - **Real impact**: One setting makes data globally accessible

5. **📁 Folder Inheritance** - Set permissions once on folders, files inherit automatically
   - **Why it matters**: Massive efficiency gains and consistent security
   - **Real impact**: Configure 100 files with 1 operation instead of 100

6. **🎯 Nearest-Node Algorithm** - The core algorithm that finds the "closest" permissions
   - **Why it matters**: Predictable, debuggable permission resolution
   - **Real impact**: No surprising permission combinations, behavior matches mental models

7. **🔍 Debugging Tools** - Trace WHY permissions work or don't work
   - **Why it matters**: Essential for troubleshooting complex scenarios
   - **Real impact**: Turn hours of debugging into minutes of systematic analysis

### 🧠 **Mental Models to Remember**

- **Higher permissions include lower ones** - Admin can do everything, Write includes Read+Create
- **Closest permissions win** - The nearest-node algorithm prevents confusion
- **Folders are powerful** - Set permissions on directories to control many files at once
- **YAML is your friend** - When in doubt, look at the generated files to understand what's happening

### 🚀 **Next Up: Part 2 - Patterns and Matching**

Now that you understand the fundamentals, Part 2 will unlock the true power of SyftBox:

- **🎯 Glob Patterns** - Use `*.py`, `docs/**`, `**/*.txt` to match multiple files with one rule
- **🏆 Pattern Specificity** - Learn how SyftBox resolves conflicts when multiple patterns match
- **🔍 Advanced Matching** - Master wildcards, directory patterns, and recursive matching
- **🐛 Pattern Debugging** - Tools to understand which patterns matched which files

**The pattern system transforms SyftBox from "good" to "incredibly powerful"** - you'll be able to create sophisticated permission schemes with just a few lines of configuration.

### 💡 **Pro Tip for Right Now**

Before moving to Part 2, try applying these concepts to a real project:
1. Create a folder structure that mirrors one of your actual projects
2. Set up realistic team permissions using the hierarchy
3. Use the debugging tools to verify everything works as expected

This hands-on practice will solidify the concepts and prepare you for the advanced patterns ahead!

In [ ]:
# Cleanup
shutil.rmtree(tutorial_dir)
print("Tutorial 1 complete! 🎉")